In [ ]:
# 디베이트 전까지 테스트

In [1]:
# 맨 위 셀에 넣기
import sys
from pathlib import Path

# capstone_project 폴더를 찾아 sys.path 맨 앞에 추가
p = Path.cwd()
root = None
for q in [p, *p.parents]:
    if q.name == "capstone_project":
        root = q
        break

if root is None:
    # 노트북이 전혀 다른 곳에서 열렸다면 경로를 직접 지정
    root = Path(r"C:\Users\ska69\capstone_project")

sys.path.insert(0, str(root))
print("PROJECT ROOT:", root)

# 테스트
import prompts
from agents.technical_agent import TechnicalAgent
print("imports ok")


PROJECT ROOT: C:\Users\ska69\capstone_project
imports ok


In [2]:
import os

os.environ["CAPSTONE_OPENAI_API"] = ""


In [3]:
# 0) 경로 가드
import sys, os
print("PYTHONPATH head:", sys.path[:2])

PYTHONPATH head: ['C:\\Users\\ska69\\capstone_project', 'C:\\Users\\ska69\\anaconda3\\python312.zip']


In [4]:
from agents.technical_agent import TechnicalAgent
import torch, json, os

# 1. 에이전트 생성
agent = TechnicalAgent(agent_id = "TechnicalAgent", ticker="NVDA", verbose=True)


In [5]:
agent

TechnicalAgent(
  (lstm1): LSTM(13, 64, batch_first=True)
  (lstm2): LSTM(64, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (drop): Dropout(p=0.18778570103014075, inplace=False)
  (loss_fn): HuberLoss()
)

In [6]:
# 2. 데이터 수집 및 지표 계산(5개년) → yfinance에서 5년 데이터 다시 받음 + 13개 TECH 지표 생성
X = agent.searcher(rebuild=True)

⚙️ NVDA TechnicalAgent dataset not found. Building new dataset...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[MacroSentiAgent] X_seq: (1222, 14, 13), y_seq: (1222, 1)
✅ NVDA MacroSentiAgent dataset saved to CSV (1222 samples, 13 features)
[SentimentalAgent] X_seq: (1196, 40, 8), y_seq: (1196, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1196 samples, 8 features)
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


In [7]:
X.shape

torch.Size([1, 55, 13])

In [8]:
# 3. 모델 학습
# 5년 데이터 기반 LSTM 학습 + 스케일러 저장 + 가중치 저장
agent.pretrain()

[23:30:34] Pretraining TechnicalAgent
  Epoch 005 | Loss: 0.383818
  Epoch 010 | Loss: 0.382168
  Epoch 015 | Loss: 0.387755
  Epoch 020 | Loss: 0.398678
  Epoch 025 | Loss: 0.398434
  Epoch 030 | Loss: 0.389612
  Epoch 035 | Loss: 0.386084
  Epoch 040 | Loss: 0.387401
  Epoch 045 | Loss: 0.393362
 TechnicalAgent 모델 학습 및 저장 완료: models\NVDA_TechnicalAgent.pt


In [9]:
# 4. 최신 윈도우 확보 → 예측 → 설명 생성 → 저장
from config.agents import dir_info

# 최신 윈도우
X_last = agent.searcher(agent.ticker)      # (1,T,F)
dates  = getattr(agent.stockdata, f"{agent.agent_id}_dates", [])


■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


In [10]:
print(X_last.shape)
print(dates[:5])

torch.Size([1, 55, 13])
['2025-08-13', '2025-08-14', '2025-08-15', '2025-08-18', '2025-08-19']


In [11]:
from core.technical_classes.technical_base_agent import Target
print(Target.__annotations__)

{'next_close': 'float', 'uncertainty': 'float | None', 'confidence': 'float | None', 'idea': 'dict | None'}


In [12]:
# 5) 예측 + 불확실성
tgt = agent.predict(X_last) 

In [13]:
# 6) 저장
from core.technical_classes.technical_base_agent import save_explain_json, load_explain_json

save_explain_json(agent.ticker, agent.agent_id, tgt)

'models/explain\\NVDA_TechnicalAgent_explain.json'

In [14]:
# 6. LLM 메시지 구성 (system, user)
sys_text, user_text = agent._build_messages_opinion(agent.stockdata, tgt)
print(sys_text[:400], "\n---\n", user_text[:700])

■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
너는 '기술적 분석 전문가'다. 입력 컨텍스트(ctx, JSON)를 기반으로
            다음날 종가 예측이 타당한 이유만 한국어로 작성하라.
            출력은 반드시 JSON 객체 {{"reason": string}} 하나만.
            예측 재계산 및 새로운 수치 생성 금지(단, 제공된 수치의 곱을 이용한 영향 점수 표기는 허용).
            개발자 용어 사용 금지: attention/어텐션, SHAP/샤프, feature 중요도/피처 중요도, Grad×Input, Occlusion, 게이트, 신뢰도, 베타, 알파 등.
            항상 사용자 중심 표현을 사용하고, ctx에 주어진 수치만 활용하라. 
---
 ctx(JSON):
        {"ticker": "NVDA", "last_price": 202.49, "next_close": 202.7342, "uncertainty": 0.0053, "confidence": 0.9947, "sigma": 0.0053, "beta": 0.9947, "window_size": 55, "idea": {"per_time": [{"date": "2025-08-13", "sum_abs": [0.019637097197139387, 0.020698133684085313, 0.021824586684691342, 0.021674376689065036, 0.019647058079641868, 0.01866797094483822, 0.017586161178408943, 0.019773929144948754, 0.02056760648828544, 0.021543489171469962, 0.021633254354999586, 0.020222648588856005, 0.016205031928880884, 0.016981575217458286, 0.016810643292115783, 0.01642855497

In [21]:
# 7. LLM 호출 및 응답 확인
op = agent.reviewer_draft(agent.stockdata, tgt)
print(f"예측 종가: {op.target.next_close:.4f}")
print("설명:", op.reason)

■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
예측 종가: 202.7342
설명: 1) vol_20d/2025-09-02 (0.3458×0.0368=0.0127): 20일 거래량은 주식의 거래 활발함을 나타내며, 거래량 증가는 가격 변동의 신뢰도를 높입니다. 2) obv/2025-09-02 (0.3444×0.0368=0.0127): 누적거래량지표는 매수·매도세를 반영해 가격 추세의 지속 가능성을 보여줍니다. 3) vol_ma_20/2025-09-02 (0.3083×0.0368=0.0113): 20일 이동평균 거래량은 거래량의 평균적 변화를 나타내며, 추세 강도를 판단하는 데 유용합니다. 4) ret_3d/2025-09-02 (0.2986×0.0368=0.0110): 3일 수익률은 단기 가격 변동성을 반영해 단기 추세를 파악하는 데 도움을 줍니다. 2025-09-02를 중심으로 한 기간은 거래량과 수익률 지표들이 동시에 높은 기여도를 보이며, 이는 거래 활발과 상방 추세가 공존하는 시기임을 의미합니다. 영향 점수에서 vol_20d와 obv가 거의 동일하게 높아 거래량과 매수세가 강력한 상승 신호임을 알 수 있고, vol_ma_20과 ret_3d도 근접한 점수로 단기 추세의 신뢰성을 더합니다. 반면, 과열 신호로 볼 수 있는 adx_14(0.2684×0.0368=0.0099)와 weekofyear_cos(0.2648×0.0368=0.0097)의 영향 점수는 상대적으로 낮아 과열이나 되돌림 가능성은 제한적임을 시사합니다. 이러한 분석을 종합하면, 거래량과 매수세가 강하게 지지하는 가운데 단기 수익률도 긍정적 변화를 보여 다음날 종가 예측이 신뢰할 만하며, 상승 방향과 안정성이 높다고 판단됩니다.


In [ ]:
# debate agent 포함 test

In [1]:
# 맨 위 셀에 넣기
import sys
from pathlib import Path

# capstone_project 폴더를 찾아 sys.path 맨 앞에 추가
p = Path.cwd()
root = None
for q in [p, *p.parents]:
    if q.name == "capstone_project":
        root = q
        break

if root is None:
    # 노트북이 전혀 다른 곳에서 열렸다면 경로를 직접 지정
    root = Path(r"C:\Users\ska69\capstone_project")

sys.path.insert(0, str(root))
print("PROJECT ROOT:", root)

# 테스트
import prompts
from agents.technical_agent import TechnicalAgent
print("imports ok")


PROJECT ROOT: C:\Users\ska69\capstone_project
imports ok


In [2]:
import os, sys, types
proj = r"C:\Users\ska69\capstone_project"  # 본인 경로
if proj not in sys.path:
    sys.path.insert(0, proj)

# macro_llm가 요구하는 agents.dump_keys 더미 모듈 주입
m = types.ModuleType("agents.dump_keys") #에러가 났어서 임의로 설정(아연)
m.CAPSTONE_OPENAI_API = ""  # 키 비워두면 됨. Macro를 호출하지 않을 것이라 문제 없음.
sys.modules["agents.dump_keys"] = m

In [3]:
import os

os.environ["CAPSTONE_OPENAI_API"] = ""


In [31]:
# 우선 테크니컬 debatagent 잘 돌아가는지 확인

from agents.debate_agent import DebateAgent

ticker = "NVDA"
da = DebateAgent(rounds=0, ticker=ticker)

USE_MACRO = False   # 매크로 포함 테스트
TECH_ONLY = True  # 테크니컬만 테스트하려면 True로

if TECH_ONLY:
    da.agents = {"TechnicalAgent": da.agents["TechnicalAgent"]}
elif USE_MACRO:
    da.agents = {
        "TechnicalAgent": da.agents["TechnicalAgent"],
        "MacroSentiAgent": da.agents["MacroSentiAgent"],
    }
else:
    da.agents = {"TechnicalAgent": da.agents["TechnicalAgent"]}

list(da.agents.keys())


['TechnicalAgent']

In [32]:
# 테크니컬 에이전트만 확인

ops = da.get_opinion(
    round=0,
    ticker=ticker,
    rebuild=True,          # 데이터 재빌드
    force_pretrain=True   # 가중치 없으면 내부 pretrain 실행
)
ops.keys()

[TechnicalAgent] searcher 실행
⚙️ NVDA TechnicalAgent rebuild requested. Building dataset...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[MacroSentiAgent] X_seq: (1222, 14, 13), y_seq: (1222, 1)
✅ NVDA MacroSentiAgent dataset saved to CSV (1222 samples, 13 features)
[SentimentalAgent] X_seq: (1196, 40, 8), y_seq: (1196, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1196 samples, 8 features)
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
[TechnicalAgent] pretrain 실행
[01:57:42] Pretraining TechnicalAgent
  Epoch 005 | Loss: 0.380661
  Epoch 010 | Loss: 0.386401
  Epoch 015 | Loss: 0.392477
  Epoch 020 | Loss: 0.398613
  Epoch 025 | Loss: 0.393584
  Epoch 030 | Loss: 0.383825
  Epoch 035 | Loss: 0.382469
  Epoch 040 | Loss: 0.383977
  Epoch 045 | Loss: 0.385993
 TechnicalAgent 모델 학습 및 저장 완료: models\NVDA_TechnicalAgent.pt
[TechnicalAgent] predict 실행
[TechnicalAgent] reviewer_draft 실행
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
  - TechnicalAgent: next_close=202.6408
 Round 0 의견 수집 완료 (1 agents)


dict_keys(['TechnicalAgent'])

In [33]:
def show(op):
    print("agent_id:", op.agent_id)
    print("next_close:", float(op.target.next_close))
    print("uncertainty:", float(op.target.uncertainty))
    print("confidence:", float(op.target.confidence))
    print("\nreason:\n", op.reason)

if "TechnicalAgent" in ops:
    print("=== Technical ===")
    show(ops["TechnicalAgent"])


=== Technical ===
agent_id: TechnicalAgent
next_close: 202.64079438105622
uncertainty: 0.003481748979538679
confidence: 0.9965363369355066

reason:
 1) obv/2025-08-13 (0.1091×0.0302=0.0033): OBV는 거래량과 가격 변동을 결합해 매수세를 파악하는 지표로, 상승 추세 확인에 유용합니다. 2) vol_20d/2025-08-13 (0.0818×0.0302=0.0025): 20일 거래량은 시장의 활발함을 나타내며, 거래 증가 시 가격 상승 가능성이 큽니다. 3) adx_14/2025-08-13 (0.0358×0.0302=0.0011): ADX는 추세 강도를 보여주며, 강한 추세일수록 가격 예측 신뢰도가 높아집니다. 4) ret_3d/2025-08-13 (0.0320×0.0302=0.0010): 3일 수익률은 단기 가격 움직임을 반영해 단기 예측에 도움을 줍니다. 2025-08-13~2025-08-14 기간은 거래량과 매수세가 모두 활발해 상방 추세가 강화된 시기로 판단됩니다. 첫 번째 지표인 OBV가 가장 높은 영향 점수를 기록해 매수세가 강함을 시사하며, 두 번째 거래량 지표도 이를 뒷받침해 거래 활발함이 가격 상승에 긍정적 영향을 미쳤습니다. ADX와 3일 수익률도 추세 강도와 단기 움직임을 반영해 예측 신뢰도를 높였으나, 영향 점수는 상대적으로 낮아 보조적 역할에 그쳤습니다. 반면, 일부 기간의 과열 신호는 낮은 영향 점수(예: 0.0001~0.0005)로 제한적 영향을 보여 예측의 안정성을 해치지 않았습니다. 종합하면, 매수세와 거래량 증가가 뚜렷한 2025-08-13~2025-08-14 기간의 흐름과 강한 추세 신호를 기반으로 다음날 종가 예측은 방향성과 안정성 측면에서 매우 타당하다고 판단됩니다.


In [37]:
ta = da.agents["TechnicalAgent"]
print(ta)

op = da.opinions[0]["TechnicalAgent"]
print(f"예측 종가: {op.target.next_close:.4f}")
print("설명:", op.reason)

TechnicalAgent(
  (lstm1): LSTM(13, 64, batch_first=True)
  (lstm2): LSTM(64, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (drop): Dropout(p=0.18778570103014075, inplace=False)
  (loss_fn): HuberLoss()
)
예측 종가: 202.6408
설명: 1) obv/2025-08-13 (0.1091×0.0302=0.0033): OBV는 거래량과 가격 변동을 결합해 매수세를 파악하는 지표로, 상승 추세 확인에 유용합니다. 2) vol_20d/2025-08-13 (0.0818×0.0302=0.0025): 20일 거래량은 시장의 활발함을 나타내며, 거래 증가 시 가격 상승 가능성이 큽니다. 3) adx_14/2025-08-13 (0.0358×0.0302=0.0011): ADX는 추세 강도를 보여주며, 강한 추세일수록 가격 예측 신뢰도가 높아집니다. 4) ret_3d/2025-08-13 (0.0320×0.0302=0.0010): 3일 수익률은 단기 가격 움직임을 반영해 단기 예측에 도움을 줍니다. 2025-08-13~2025-08-14 기간은 거래량과 매수세가 모두 활발해 상방 추세가 강화된 시기로 판단됩니다. 첫 번째 지표인 OBV가 가장 높은 영향 점수를 기록해 매수세가 강함을 시사하며, 두 번째 거래량 지표도 이를 뒷받침해 거래 활발함이 가격 상승에 긍정적 영향을 미쳤습니다. ADX와 3일 수익률도 추세 강도와 단기 움직임을 반영해 예측 신뢰도를 높였으나, 영향 점수는 상대적으로 낮아 보조적 역할에 그쳤습니다. 반면, 일부 기간의 과열 신호는 낮은 영향 점수(예: 0.0001~0.0005)로 제한적 영향을 보여 예측의 안정성을 해치지 않았습니다. 종합하면, 매수세와 거래량 증가가 뚜렷한 2025-08-13~2025-08-14 기간의 흐름과 강한 추세 신